# Libraries and Drive

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
import os
os.chdir('/content/drive/My Drive/Volatility_forecasting/')

Mounted at /content/drive


In [2]:
!pip install neuralforecast dask[dataframe]
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.8/254.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/7

# Modelling original (20 years total, 14 train, 2 val, 4 test)

In [ ]:
from neuralforecast.core import NeuralForecast
from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch
from neuralforecast.losses.pytorch import DistributionLoss
from neuralforecast.auto import  AutoNHITS

In [ ]:
df = pd.read_csv('Data/S&P500/3ProSP500.csv')
df['ds'] = pd.to_datetime(df['ds'])
df = df.astype({col: 'int32' if dtype == 'int64' else 'float32' if dtype == 'float64' else dtype
                for col, dtype in df.dtypes.items()})
df = df.rename(columns={'840E': 'y'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008545 entries, 0 to 2008544
Data columns (total 21 columns):
 #   Column     Dtype         
---  ------     -----         
 0   unique_id  object        
 1   ds         datetime64[ns]
 2   y          float32       
 3   MA(2,9)    int32         
 4   MA(3,9)    int32         
 5   MA(1,12)   int32         
 6   MA(2,12)   int32         
 7   MA(3,12)   int32         
 8   MOM(9)     int32         
 9   MOM(12)    int32         
 10  RSI(7)     int32         
 11  RSI(14)    int32         
 12  EMA(3,9)   int32         
 13  EMA(5,9)   int32         
 14  EMA(5,12)  int32         
 15  DY         float32       
 16  PTBV       float32       
 17  P          float32       
 18  PO         float32       
 19  VO         float32       
 20  PE         float32       
dtypes: datetime64[ns](1), float32(7), int32(12), object(1)
memory usage: 176.2+ MB


In [ ]:
start_2020_2023 = '2020-01-01'
end_2020_2023 = '2023-12-31'

start_2018_2019 = '2018-01-01'
end_2018_2019 = '2019-12-31'

# Filter the DataFrame for each time range and extract unique days
unique_days_2020_2023 = df[(df['ds'] >= start_2020_2023) & (df['ds'] <= end_2020_2023)]['ds'].dt.date.unique()
unique_days_2018_2019 = df[(df['ds'] >= start_2018_2019) & (df['ds'] <= end_2018_2019)]['ds'].dt.date.unique()

# Count the number of unique days in each range
count_unique_days_2020_2023 = len(unique_days_2020_2023)
count_unique_days_2018_2019 = len(unique_days_2018_2019)

# Create a summary DataFrame
summary_counts = pd.DataFrame({
    'Period': ['2020-2023', '2018-2019'],
    'Unique Days Covered': [count_unique_days_2020_2023, count_unique_days_2018_2019]
})
summary_counts

,Period,Unique Days Covered
0,2020-2023,1043
1,2018-2019,522


In [ ]:
import os
from time import time

class AutoNHITSTrainer:
    def __init__(self, horizons, levels, exog_list, df, val_size, test_size):
        self.horizons = horizons
        self.levels = levels
        self.exog_list = exog_list
        self.df = df
        self.val_size = val_size
        self.test_size = test_size

    def check_existing_files(self, horizon):
        """Checks if model and CSV already exist for a given horizon."""
        model_path = f'Trained Models/AutoNHITS/horizon_{horizon}/'
        output_csv = f'Data/Test/nhits_model0_1_horizon_{horizon}.csv'
        return os.path.exists(model_path) and os.path.exists(output_csv)

    def save_results(self, nf, horizon, Y_hat_df):
        """Saves the trained model and prediction results."""
        model_path = f'Trained Models/AutoNHITS/horizon_{horizon}/'
        output_csv = f'Data/Test/horizon_{horizon}/nhits_model0_1_horizon_{horizon}.csv'

        # Create model directory if it doesn't exist
        os.makedirs(model_path, exist_ok=True)

        # Save the model, predictions and hyperparameter search
        for idx, model in enumerate(nf.models):
          hpo = f'HPO/horizon_{horizon}/nhits_model{idx}_horizon_{horizon}_hpo.csv'
          results = model.results.get_dataframe()
          results.to_csv(hpo, index=False)

        nf.save(path=model_path, model_index=None, overwrite=True, save_dataset=False)

        for col in Y_hat_df.select_dtypes(include='float32').columns:
          Y_hat_df[col] = Y_hat_df[col].astype('float16')

        Y_hat_df.to_csv(output_csv, index=False)


    def configure_models(self, horizon):
        """Configures two AutoNHITS models for the given horizon."""
        # Model 0 Configuration
        nhits_config0 = AutoNHITS.get_default_config(h=horizon, backend="ray")
        nhits_config0['random_seed'] = 42
        nhits_config0['learning_rate'] = tune.choice([0.01, 0.005, 0.001, 0.0005, 0.0001, 0.0005, 0.0001, 0.00005, 0.00001])

        # Model 1 Configuration
        nhits_config1 = AutoNHITS.get_default_config(h=horizon, backend="ray")
        nhits_config1['hist_exog_list'] = self.exog_list
        nhits_config1['random_seed'] = 42
        nhits_config1['learning_rate'] = tune.choice([0.01, 0.005, 0.001, 0.0005, 0.0001, 0.0005, 0.0001, 0.00005, 0.00001])

        # Instantiate the models
        nhits_model0 = AutoNHITS(h=horizon,
                                 config=nhits_config0,
                                 search_alg=HyperOptSearch(), num_samples=30,
                                 backend='ray',
                                 loss=DistributionLoss(distribution='StudentT', level=self.levels),
                                 cpus = 12, gpus = 1)

        nhits_model1 = AutoNHITS(h=horizon,
                                 config=nhits_config1,
                                 search_alg=HyperOptSearch(), num_samples=30,
                                 backend='ray',
                                 loss=DistributionLoss(distribution='StudentT', level=self.levels),
                                 cpus = 12, gpus = 1)

        return nhits_model0, nhits_model1

    def run_training(self):
        """Runs the training loop over all horizons."""
        for horizon in self.horizons:
            if self.check_existing_files(horizon):
                print(f"Horizon {horizon}: Model and CSV already exist. Skipping this run.")
                continue

            # Configure the models
            nhits_model0, nhits_model1 = self.configure_models(horizon)

            # Start training and cross-validation
            init = time()
            nf = NeuralForecast(models=[nhits_model0, nhits_model1], freq='B')
            Y_hat_df = nf.cross_validation(df=self.df,
                                           val_size=self.val_size,
                                           test_size=self.test_size,
                                           n_windows=None)

            # Save results
            self.save_results(nf, horizon, Y_hat_df)

            # Log the time taken
            end = time()
            print(f'Horizon {horizon} CV Minutes: {(end - init) / 60}')

In [ ]:
exog_list = list(df.columns)
exog_list.remove('ds')
exog_list.remove('y')
exog_list.remove('unique_id')

levels = [90]
val_size = count_unique_days_2018_2019
test_size = count_unique_days_2020_2023
horizons = [1, 5, 10]

In [ ]:
trainer = AutoNHITSTrainer(horizons, levels, exog_list, df, val_size, test_size)
trainer.run_training()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
exog_list = list(df.columns)
exog_list.remove('ds')
exog_list.remove('y')
exog_list.remove('unique_id')

levels = [90]
val_size = count_unique_days_2018_2019
test_size = count_unique_days_2020_2023
horizons = [20]

In [ ]:
trainer = AutoNHITSTrainer(horizons, levels, exog_list, df, val_size, test_size)
trainer.run_training()

2024-11-12 18:17:40,208	INFO worker.py:1816 -- Started a local Ray instance.
2024-11-12 18:17:41,607	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


+--------------------------------------------------------------------+
| Configuration for experiment     _train_tune_2024-11-12_18-17-38   |
+--------------------------------------------------------------------+
| Search algorithm                 SearchGenerator                   |
| Scheduler                        FIFOScheduler                     |
| Number of trials                 30                                |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/_train_tune_2024-11-12_18-17-38
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-11-12_18-17-38_352630_2326/artifacts/2024-11-12_18-17-41/_train_tune_2024-11-12_18-17-38/driver_artifacts`


(_train_tune pid=3770) /usr/local/lib/python3.10/dist-packages/ray/tune/integration/pytorch_lightning.py:198: `ray.tune.integration.pytorch_lightning.TuneReportCallback` is deprecated. Use `ray.tune.integration.pytorch_lightning.TuneReportCheckpointCallback` instead.
(_train_tune pid=3770) Seed set to 42
(_train_tune pid=3770) GPU available: True (cuda), used: True
(_train_tune pid=3770) TPU available: False, using: 0 TPU cores
(_train_tune pid=3770) HPU available: False, using: 0 HPUs
(_train_tune pid=3770) You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
(_train_tune pid=3770) 2024-11-12 18:17:49.815176: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Epoch 7:   0%|          | 0/13 [00:00<?, ?it/s, v_num=0, train_loss_step=-1.12, train_loss_epoch=-1.11]
(_train_tune pid=3770) 
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/13 [00:00<?, ?it/s]
(_train_tune pid=3770) 
Validation DataLoader 0: 100%|██████████| 13/13 [00:01<00:00,  7.43it/s]
(_train_tune pid=3770) 
Epoch 15:   0%|          | 0/13 [00:00<?, ?it/s, v_num=0, train_loss_step=-0.848, train_loss_epoch=-1.12, valid_loss=-1.18]
(_train_tune pid=3770) 
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/13 [00:00<?, ?it/s]
(_train_tune pid=3770) 
Epoch 23:   0%|          | 0/13 [00:00<?, ?it/s, v_num=0, train_loss_step=-1.17, train_loss_epoch=-1.15, valid_loss=-1.20]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/13 [00:00<?, ?it/s]
(_train_tune pid=3770) 
Epoch 30:   0%|          | 0/13 [00:00<?, ?it/s, v_

(_train_tune pid=3770) `Trainer.fit` stopped: `max_steps=1100.0` reached.
2024-11-12 18:18:42,416	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'loss': ('__ref_ph', 'de895953'), 'n_freq_downsample': (180, 60, 1), 'n_pool_kernel_size': (16, 8, 1), 'valid_loss': ('__ref_ph', '004b9a7a')}


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=900.0` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Horizon 20 CV Minutes: 73.07346337238947


# Modelling half (12 years total, 6 train, 2 val, 4 test)

In [ ]:
from neuralforecast.core import NeuralForecast
from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch
from neuralforecast.losses.pytorch import DistributionLoss
from neuralforecast.auto import  AutoNHITS

In [ ]:
df = pd.read_csv('Data/S&P500/3ProSP500.csv')
df['ds'] = pd.to_datetime(df['ds'])
df = df.astype({col: 'int32' if dtype == 'int64' else 'float32' if dtype == 'float64' else dtype
                for col, dtype in df.dtypes.items()})
df = df.rename(columns={'840E': 'y'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008545 entries, 0 to 2008544
Data columns (total 21 columns):
 #   Column     Dtype         
---  ------     -----         
 0   unique_id  object        
 1   ds         datetime64[ns]
 2   y          float32       
 3   MA(2,9)    int32         
 4   MA(3,9)    int32         
 5   MA(1,12)   int32         
 6   MA(2,12)   int32         
 7   MA(3,12)   int32         
 8   MOM(9)     int32         
 9   MOM(12)    int32         
 10  RSI(7)     int32         
 11  RSI(14)    int32         
 12  EMA(3,9)   int32         
 13  EMA(5,9)   int32         
 14  EMA(5,12)  int32         
 15  DY         float32       
 16  PTBV       float32       
 17  P          float32       
 18  PO         float32       
 19  VO         float32       
 20  PE         float32       
dtypes: datetime64[ns](1), float32(7), int32(12), object(1)
memory usage: 176.2+ MB


In [ ]:
print(df['ds'].min().year, df['ds'].max().year)
print(df['ds'].max().year - df['ds'].min().year + 1)

2004 2023
20


In [ ]:
# Calculate the starting year and add 8 years
starting_year = df['ds'].min().year + 8
# Create a new date range
new_start_date = pd.to_datetime(str(starting_year) + '-' + str(df['ds'].min().month) + '-' + str(df['ds'].min().day))
#Filter out the data based on the new starting year
df = df[df['ds'] >= new_start_date]
print(df['ds'].min().year, df['ds'].max().year)
print(df['ds'].max().year - df['ds'].min().year + 1)

2012 2023
12


In [ ]:
start_2020_2023 = '2020-01-01'
end_2020_2023 = '2023-12-31'

start_2018_2019 = '2018-01-01'
end_2018_2019 = '2019-12-31'

# Filter the DataFrame for each time range and extract unique days
unique_days_2020_2023 = df[(df['ds'] >= start_2020_2023) & (df['ds'] <= end_2020_2023)]['ds'].dt.date.unique()
unique_days_2018_2019 = df[(df['ds'] >= start_2018_2019) & (df['ds'] <= end_2018_2019)]['ds'].dt.date.unique()

# Count the number of unique days in each range
count_unique_days_2020_2023 = len(unique_days_2020_2023)
count_unique_days_2018_2019 = len(unique_days_2018_2019)

# Create a summary DataFrame
summary_counts = pd.DataFrame({
    'Period': ['2020-2023', '2018-2019'],
    'Unique Days Covered': [count_unique_days_2020_2023, count_unique_days_2018_2019]
})
summary_counts

,Period,Unique Days Covered
0,2020-2023,1043
1,2018-2019,522


In [ ]:
exog_list = list(df.columns)
exog_list.remove('ds')
exog_list.remove('y')
exog_list.remove('unique_id')

levels = [90]
val_size = count_unique_days_2018_2019
test_size = count_unique_days_2020_2023
horizons = [1, 5, 10, 20]

In [ ]:
import os
from time import time

class AutoNHITSTrainer:
    def __init__(self, horizons, levels, exog_list, df, val_size, test_size):
        self.horizons = horizons
        self.levels = levels
        self.exog_list = exog_list
        self.df = df
        self.val_size = val_size
        self.test_size = test_size

    def check_existing_files(self, horizon):
        """Checks if model and CSV already exist for a given horizon."""
        model_path = f'Trained Models/AutoNHITS/8TYhorizon_{horizon}/'
        output_csv = f'Data/Test/8TYnhits_model0_1_horizon_{horizon}.csv'
        return os.path.exists(model_path) and os.path.exists(output_csv)

    def save_results(self, nf, horizon, Y_hat_df):
        """Saves the trained model and prediction results."""
        model_path = f'Trained Models/AutoNHITS/8TYhorizon_{horizon}/'
        output_csv = f'Data/Test/horizon_{horizon}/8TYnhits_model0_1_horizon_{horizon}.csv'

        # Create model directory if it doesn't exist
        os.makedirs(model_path, exist_ok=True)

        # Save the model, predictions and hyperparameter search
        for idx, model in enumerate(nf.models):
          hpo = f'HPO/horizon_{horizon}/8TYnhits_model{idx}_horizon_{horizon}_hpo.csv'
          results = model.results.get_dataframe()
          results.to_csv(hpo, index=False)

        nf.save(path=model_path, model_index=None, overwrite=True, save_dataset=False)

        for col in Y_hat_df.select_dtypes(include='float32').columns:
          Y_hat_df[col] = Y_hat_df[col].astype('float16')

        Y_hat_df.to_csv(output_csv, index=False)


    def configure_models(self, horizon):
        """Configures two AutoNHITS models for the given horizon."""
        # Model 0 Configuration
        nhits_config0 = AutoNHITS.get_default_config(h=horizon, backend="ray")
        nhits_config0['random_seed'] = 42
        nhits_config0['learning_rate'] = tune.choice([0.01, 0.005, 0.001, 0.0005, 0.0001, 0.0005, 0.0001, 0.00005, 0.00001])

        # Model 1 Configuration
        nhits_config1 = AutoNHITS.get_default_config(h=horizon, backend="ray")
        nhits_config1['hist_exog_list'] = self.exog_list
        nhits_config1['random_seed'] = 42
        nhits_config1['learning_rate'] = tune.choice([0.01, 0.005, 0.001, 0.0005, 0.0001, 0.0005, 0.0001, 0.00005, 0.00001])

        # Instantiate the models
        nhits_model0 = AutoNHITS(h=horizon,
                                 config=nhits_config0,
                                 search_alg=HyperOptSearch(), num_samples=30,
                                 backend='ray',
                                 loss=DistributionLoss(distribution='StudentT', level=self.levels),
                                 cpus = 12, gpus = 1)

        nhits_model1 = AutoNHITS(h=horizon,
                                 config=nhits_config1,
                                 search_alg=HyperOptSearch(), num_samples=30,
                                 backend='ray',
                                 loss=DistributionLoss(distribution='StudentT', level=self.levels),
                                 cpus = 12, gpus = 1)

        return nhits_model0, nhits_model1

    def run_training(self):
        """Runs the training loop over all horizons."""
        for horizon in self.horizons:
            if self.check_existing_files(horizon):
                print(f"Horizon {horizon}: Model and CSV already exist. Skipping this run.")
                continue

            # Configure the models
            nhits_model0, nhits_model1 = self.configure_models(horizon)

            # Start training and cross-validation
            init = time()
            nf = NeuralForecast(models=[nhits_model0, nhits_model1], freq='B')
            Y_hat_df = nf.cross_validation(df=self.df,
                                           val_size=self.val_size,
                                           test_size=self.test_size,
                                           n_windows=None)

            # Save results
            self.save_results(nf, horizon, Y_hat_df)

            # Log the time taken
            end = time()
            print(f'Horizon {horizon} CV Minutes: {(end - init) / 60}')

In [ ]:
trainer = AutoNHITSTrainer(horizons, levels, exog_list, df, val_size, test_size)
trainer.run_training()

+--------------------------------------------------------------------+
| Configuration for experiment     _train_tune_2024-11-13_20-14-28   |
+--------------------------------------------------------------------+
| Search algorithm                 SearchGenerator                   |
| Scheduler                        FIFOScheduler                     |
| Number of trials                 30                                |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/_train_tune_2024-11-13_20-14-28
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-11-13_11-38-37_188179_3696/artifacts/2024-11-13_20-14-28/_train_tune_2024-11-13_20-14-28/driver_artifacts`


(_train_tune pid=159411) /usr/local/lib/python3.10/dist-packages/ray/tune/integration/pytorch_lightning.py:198: `ray.tune.integration.pytorch_lightning.TuneReportCallback` is deprecated. Use `ray.tune.integration.pytorch_lightning.TuneReportCheckpointCallback` instead.
(_train_tune pid=159411) Seed set to 42
(_train_tune pid=159411) GPU available: True (cuda), used: True
(_train_tune pid=159411) TPU available: False, using: 0 TPU cores
(_train_tune pid=159411) HPU available: False, using: 0 HPUs
(_train_tune pid=159411) You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
(_train_tune pid=159411) 2024-11-13 20:14:35.819724: I tensorflow/core/util/port.cc:153] oneDNN custom operation

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Epoch 49: 100%|██████████| 2/2 [00:00<00:00, 12.60it/s, v_num=0, train_loss_step=-1.34, train_loss_epoch=-1.33]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]
(_train_tune pid=159411) 
Epoch 67:   0%|          | 0/2 [00:00<?, ?it/s, v_num=0, train_loss_step=-1.32, train_loss_epoch=-1.32, valid_loss=-1.33]


2024-11-13 20:14:51,595	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-11-13 20:14:51,607	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/_train_tune_2024-11-13_20-14-28' in 0.0098s.


Epoch 99: 100%|██████████| 2/2 [00:00<00:00, 13.34it/s, v_num=0, train_loss_step=-1.33, train_loss_epoch=-1.35, valid_loss=-1.33]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]


2024-11-13 20:14:57,057	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: Tuner.restore(path="/root/ray_results/_train_tune_2024-11-13_20-14-28", trainable=...)
2024-11-13 20:14:57,063	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 1 trial(s):
- _train_tune_d6792489: FileNotFoundError('Could not fetch metrics for _train_tune_d6792489: both result.json and progress.csv were not found at /root/ray_results/_train_tune_2024-11-13_20-14-28/_train_tune_d6792489_2_batch_size=128,h=1,input_size=2,learning_rate=0.0005,loss=ref_ph_de895953,max_steps=600.0000,n_freq_downsam_2024-11-13_20-14-35')
INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs



(_train_tune pid=159411) 
Validation DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  5.21it/s]


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | loss         | DistributionLoss | 3      | eval 
1 | padder_train | ConstantPad1d    | 0      | train
2 | scaler       | TemporalNorm     | 0      | train
3 | blocks       | ModuleList       | 2.4 M  | train
----------------------------------------------------------
2.4 M     Trainable params
3         Non-trainable params
2.4 M     Total params
9.493     Total estimated model params size (MB)
33        Modules in train mode
1         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

# Modelling Technical Indicators (20 years total, 14 train, 2 val, 4 test)

In [ ]:
from neuralforecast.core import NeuralForecast
from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch
from neuralforecast.losses.pytorch import DistributionLoss
from neuralforecast.auto import AutoNHITS

In [ ]:
df = pd.read_csv('Data/S&P500/3ProSP500.csv')
df['ds'] = pd.to_datetime(df['ds'])
df = df.astype({col: 'int32' if dtype == 'int64' else 'float32' if dtype == 'float64' else dtype
                for col, dtype in df.dtypes.items()})
df = df.rename(columns={'840E': 'y'})

selected_columns = [
    'unique_id', 'ds', 'y', 'MA(2,9)', 'MA(3,9)', 'MA(1,12)',
    'MA(2,12)', 'MA(3,12)', 'MOM(9)', 'MOM(12)', 'RSI(7)',
    'RSI(14)', 'EMA(3,9)', 'EMA(5,9)', 'EMA(5,12)'
]
df = df[selected_columns]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008545 entries, 0 to 2008544
Data columns (total 15 columns):
 #   Column     Dtype         
---  ------     -----         
 0   unique_id  object        
 1   ds         datetime64[ns]
 2   y          float32       
 3   MA(2,9)    int32         
 4   MA(3,9)    int32         
 5   MA(1,12)   int32         
 6   MA(2,12)   int32         
 7   MA(3,12)   int32         
 8   MOM(9)     int32         
 9   MOM(12)    int32         
 10  RSI(7)     int32         
 11  RSI(14)    int32         
 12  EMA(3,9)   int32         
 13  EMA(5,9)   int32         
 14  EMA(5,12)  int32         
dtypes: datetime64[ns](1), float32(1), int32(12), object(1)
memory usage: 130.3+ MB


In [ ]:
start_2020_2023 = '2020-01-01'
end_2020_2023 = '2023-12-31'

start_2018_2019 = '2018-01-01'
end_2018_2019 = '2019-12-31'

# Filter the DataFrame for each time range and extract unique days
unique_days_2020_2023 = df[(df['ds'] >= start_2020_2023) & (df['ds'] <= end_2020_2023)]['ds'].dt.date.unique()
unique_days_2018_2019 = df[(df['ds'] >= start_2018_2019) & (df['ds'] <= end_2018_2019)]['ds'].dt.date.unique()

# Count the number of unique days in each range
count_unique_days_2020_2023 = len(unique_days_2020_2023)
count_unique_days_2018_2019 = len(unique_days_2018_2019)

# Create a summary DataFrame
summary_counts = pd.DataFrame({
    'Period': ['2020-2023', '2018-2019'],
    'Unique Days Covered': [count_unique_days_2020_2023, count_unique_days_2018_2019]
})
summary_counts

,Period,Unique Days Covered
0,2020-2023,1043
1,2018-2019,522


In [ ]:
exog_list = list(df.columns)
exog_list.remove('ds')
exog_list.remove('y')
exog_list.remove('unique_id')
exog_list

['MA(2,9)',
 'MA(3,9)',
 'MA(1,12)',
 'MA(2,12)',
 'MA(3,12)',
 'MOM(9)',
 'MOM(12)',
 'RSI(7)',
 'RSI(14)',
 'EMA(3,9)',
 'EMA(5,9)',
 'EMA(5,12)']

In [ ]:
levels = [90]
val_size = count_unique_days_2018_2019
test_size = count_unique_days_2020_2023
horizons = [1, 5, 10, 20]

In [ ]:
import os
from time import time

class AutoNHITSTrainer:
    def __init__(self, horizons, levels, exog_list, df, val_size, test_size):
        self.horizons = horizons
        self.levels = levels
        self.exog_list = exog_list
        self.df = df
        self.val_size = val_size
        self.test_size = test_size

    def check_existing_files(self, horizon):
        """Checks if model and CSV already exist for a given horizon."""
        model_path = f'Trained Models/AutoNHITS/TECHhorizon_{horizon}/'
        output_csv = f'Data/Test/horizon_{horizon}/TECHnhits_model1_horizon_{horizon}.csv'
        return os.path.exists(model_path) and os.path.exists(output_csv)

    def save_results(self, nf, horizon, Y_hat_df):
        """Saves the trained model and prediction results."""
        model_path = f'Trained Models/AutoNHITS/TECHhorizon_{horizon}/'
        output_csv = f'Data/Test/horizon_{horizon}/TECHnhits_model1_horizon_{horizon}.csv'

        # Create model directory if it doesn't exist
        os.makedirs(model_path, exist_ok=True)

        # Save the model, predictions and hyperparameter search
        hpo = f'HPO/horizon_{horizon}/TECHnhits_model1_horizon_{horizon}_hpo.csv'
        results = nf.models[0].results.get_dataframe()
        results.to_csv(hpo, index=False)

        nf.save(path=model_path, model_index=None, overwrite=True, save_dataset=False)

        for col in Y_hat_df.select_dtypes(include='float32').columns:
            Y_hat_df[col] = Y_hat_df[col].astype('float16')

        Y_hat_df.to_csv(output_csv, index=False)

    def configure_model(self, horizon):
        """Configures the AutoNHITS model for the given horizon."""
        nhits_config = AutoNHITS.get_default_config(h=horizon, backend="ray")
        nhits_config['hist_exog_list'] = self.exog_list
        nhits_config['random_seed'] = 42
        nhits_config['learning_rate'] = tune.choice([0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001])

        # Instantiate the model
        nhits_model = AutoNHITS(h=horizon,
                            config=nhits_config,
                            search_alg=HyperOptSearch(), num_samples=30,
                            backend='ray',
                            loss=DistributionLoss(distribution='StudentT', level=self.levels),
                            cpus=12, gpus=1)

        return nhits_model

    def run_training(self):
        """Runs the training loop over all horizons."""
        for horizon in self.horizons:
            if self.check_existing_files(horizon):
                print(f"Horizon {horizon}: Model and CSV already exist. Skipping this run.")
                continue

            # Configure the model
            nhits_model = self.configure_model(horizon)

            # Start training and cross-validation
            init = time()
            nf = NeuralForecast(models=[nhits_model], freq='B')
            Y_hat_df = nf.cross_validation(df=self.df,
                                           val_size=self.val_size,
                                           test_size=self.test_size,
                                           n_windows=None)

            # Save results
            self.save_results(nf, horizon, Y_hat_df)

            # Log the time taken
            end = time()
            print(f'Horizon {horizon} CV Minutes: {(end - init) / 60}')

In [ ]:
# Initialize and run the trainer
trainer = AutoNHITSTrainer(horizons, levels, exog_list, df, val_size, test_size)
trainer.run_training()

Output hidden; open in https://colab.research.google.com to view.

# Modelling Firm Indicators (20 years total, 14 train, 2 val, 4 test)

In [3]:
from neuralforecast.core import NeuralForecast
from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch
from neuralforecast.losses.pytorch import DistributionLoss
from neuralforecast.auto import AutoNHITS

In [4]:
df = pd.read_csv('Data/S&P500/3ProSP500.csv')
df['ds'] = pd.to_datetime(df['ds'])
df = df.astype({col: 'int32' if dtype == 'int64' else 'float32' if dtype == 'float64' else dtype
                for col, dtype in df.dtypes.items()})
df = df.rename(columns={'840E': 'y'})

selected_columns = ['unique_id', 'ds', 'y', 'DY',  'PTBV',  'P',  'PO',  'VO',  'PE']
df = df[selected_columns]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008545 entries, 0 to 2008544
Data columns (total 9 columns):
 #   Column     Dtype         
---  ------     -----         
 0   unique_id  object        
 1   ds         datetime64[ns]
 2   y          float32       
 3   DY         float32       
 4   PTBV       float32       
 5   P          float32       
 6   PO         float32       
 7   VO         float32       
 8   PE         float32       
dtypes: datetime64[ns](1), float32(7), object(1)
memory usage: 84.3+ MB


In [5]:
print(df['ds'].min().year, df['ds'].max().year)
print(df['ds'].max().year - df['ds'].min().year + 1)

2004 2023
20


In [6]:
start_2020_2023 = '2020-01-01'
end_2020_2023 = '2023-12-31'

start_2018_2019 = '2018-01-01'
end_2018_2019 = '2019-12-31'

# Filter the DataFrame for each time range and extract unique days
unique_days_2020_2023 = df[(df['ds'] >= start_2020_2023) & (df['ds'] <= end_2020_2023)]['ds'].dt.date.unique()
unique_days_2018_2019 = df[(df['ds'] >= start_2018_2019) & (df['ds'] <= end_2018_2019)]['ds'].dt.date.unique()

# Count the number of unique days in each range
count_unique_days_2020_2023 = len(unique_days_2020_2023)
count_unique_days_2018_2019 = len(unique_days_2018_2019)

# Create a summary DataFrame
summary_counts = pd.DataFrame({
    'Period': ['2020-2023', '2018-2019'],
    'Unique Days Covered': [count_unique_days_2020_2023, count_unique_days_2018_2019]
})
summary_counts

,Period,Unique Days Covered
0,2020-2023,1043
1,2018-2019,522


In [7]:
exog_list = list(df.columns)
exog_list.remove('ds')
exog_list.remove('y')
exog_list.remove('unique_id')
exog_list

['DY', 'PTBV', 'P', 'PO', 'VO', 'PE']

In [8]:
levels = [90]
val_size = count_unique_days_2018_2019
test_size = count_unique_days_2020_2023
horizons = [1, 5, 10, 20]

In [9]:
import os
from time import time

class AutoNHITSTrainer:
    def __init__(self, horizons, levels, exog_list, df, val_size, test_size):
        self.horizons = horizons
        self.levels = levels
        self.exog_list = exog_list
        self.df = df
        self.val_size = val_size
        self.test_size = test_size

    def check_existing_files(self, horizon):
        """Checks if model and CSV already exist for a given horizon."""
        model_path = f'Trained Models/AutoNHITS/FIRMhorizon_{horizon}/'
        output_csv = f'Data/Test/horizon_{horizon}/FIRMnhits_model1_horizon_{horizon}.csv'
        return os.path.exists(model_path) and os.path.exists(output_csv)

    def save_results(self, nf, horizon, Y_hat_df):
        """Saves the trained model and prediction results."""
        model_path = f'Trained Models/AutoNHITS/FIRMhorizon_{horizon}/'
        output_csv = f'Data/Test/horizon_{horizon}/FIRMnhits_model1_horizon_{horizon}.csv'

        # Create model directory if it doesn't exist
        os.makedirs(model_path, exist_ok=True)

        # Save the model, predictions and hyperparameter search
        hpo = f'HPO/horizon_{horizon}/FIRMnhits_model1_horizon_{horizon}_hpo.csv'
        results = nf.models[0].results.get_dataframe()
        results.to_csv(hpo, index=False)

        nf.save(path=model_path, model_index=None, overwrite=True, save_dataset=False)

        for col in Y_hat_df.select_dtypes(include='float32').columns:
            Y_hat_df[col] = Y_hat_df[col].astype('float16')

        Y_hat_df.to_csv(output_csv, index=False)

    def configure_model(self, horizon):
        """Configures the AutoNHITS model for the given horizon."""
        nhits_config = AutoNHITS.get_default_config(h=horizon, backend="ray")
        nhits_config['hist_exog_list'] = self.exog_list
        nhits_config['random_seed'] = 42
        nhits_config['learning_rate'] = tune.choice([0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001])

        # Instantiate the model
        nhits_model = AutoNHITS(h=horizon,
                            config=nhits_config,
                            search_alg=HyperOptSearch(), num_samples=30,
                            backend='ray',
                            loss=DistributionLoss(distribution='StudentT', level=self.levels),
                            cpus=12, gpus=1)

        return nhits_model

    def run_training(self):
        """Runs the training loop over all horizons."""
        for horizon in self.horizons:
            if self.check_existing_files(horizon):
                print(f"Horizon {horizon}: Model and CSV already exist. Skipping this run.")
                continue

            # Configure the model
            nhits_model = self.configure_model(horizon)

            # Start training and cross-validation
            init = time()
            nf = NeuralForecast(models=[nhits_model], freq='B')
            Y_hat_df = nf.cross_validation(df=self.df,
                                           val_size=self.val_size,
                                           test_size=self.test_size,
                                           n_windows=None)

            # Save results
            self.save_results(nf, horizon, Y_hat_df)

            # Log the time taken
            end = time()
            print(f'Horizon {horizon} CV Minutes: {(end - init) / 60}')

In [ ]:
# Initialize and run the trainer
trainer = AutoNHITSTrainer(horizons, levels, exog_list, df, val_size, test_size)
trainer.run_training()

Horizon 1: Model and CSV already exist. Skipping this run.
Horizon 5: Model and CSV already exist. Skipping this run.
Horizon 10: Model and CSV already exist. Skipping this run.


2024-11-18 13:03:36,165	INFO worker.py:1819 -- Started a local Ray instance.
2024-11-18 13:03:37,350	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


+--------------------------------------------------------------------+
| Configuration for experiment     _train_tune_2024-11-18_13-03-34   |
+--------------------------------------------------------------------+
| Search algorithm                 SearchGenerator                   |
| Scheduler                        FIFOScheduler                     |
| Number of trials                 30                                |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/_train_tune_2024-11-18_13-03-34
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-11-18_13-03-34_336407_4396/artifacts/2024-11-18_13-03-37/_train_tune_2024-11-18_13-03-34/driver_artifacts`


(_train_tune pid=5856) /usr/local/lib/python3.10/dist-packages/ray/tune/integration/pytorch_lightning.py:198: `ray.tune.integration.pytorch_lightning.TuneReportCallback` is deprecated. Use `ray.tune.integration.pytorch_lightning.TuneReportCheckpointCallback` instead.
(_train_tune pid=5856) Seed set to 42
(_train_tune pid=5856) GPU available: True (cuda), used: True
(_train_tune pid=5856) TPU available: False, using: 0 TPU cores
(_train_tune pid=5856) HPU available: False, using: 0 HPUs
(_train_tune pid=5856) You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
(_train_tune pid=5856) 2024-11-18 13:03:45.302446: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Epoch 7:   0%|          | 0/13 [00:00<?, ?it/s, v_num=0, train_loss_step=-0.842, train_loss_epoch=-1.15]
(_train_tune pid=5856) 
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/13 [00:00<?, ?it/s]
(_train_tune pid=5856) 
Validation DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 60.14it/s]
(_train_tune pid=5856) 
Epoch 15:   0%|          | 0/13 [00:00<?, ?it/s, v_num=0, train_loss_step=-0.963, train_loss_epoch=-1.22, valid_loss=-1.14]
Validation: |          | 0/? [00:00<?, ?it/s]
(_train_tune pid=5856) 
Validation DataLoader 0:   0%|          | 0/13 [00:00<?, ?it/s]
(_train_tune pid=5856) 
Epoch 23:   0%|          | 0/13 [00:00<?, ?it/s, v_num=0, train_loss_step=-0.885, train_loss_epoch=-1.18, valid_loss=-1.17]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/13 [00:00<?, ?it/s]
(_train_tune pid=5856) 
Epoch 30:   0%|          | 0/13 [00:00<?, ?it/s, 

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1200.0` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Horizon 20 CV Minutes: 24.180792804559072
